In [11]:
# 최신순으로 최근 50개 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [1]:
#!pip install selenium

     ---------------------------------------- 6.3/6.3 MB 15.5 MB/s eta 0:00:00
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
     ------------------------------------- 384.9/384.9 kB 12.1 MB/s eta 0:00:00
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time

In [3]:
cafeName = 'Auspice coffee'
cafeNum = '0507-1305-0628'
cafeID = ""

In [4]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [5]:
cafeID = findCafe(cafeNum, cafeName)
cafeID

'1143065220'

In [6]:
finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'


In [7]:
driver = webdriver.Chrome()
driver.get(finalurl)

In [8]:
i=0
while i<5: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP) 
            time.sleep(3) 
            
            driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click() 
            time.sleep(3)
            i+=1
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 

In [9]:
#!pip install lxml

In [10]:
html = driver.page_source 
soup = BeautifulSoup(html, 'lxml')

In [11]:
one_review = soup.select('.YeINN')
review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
print('리뷰 총 개수 : '+str(review_num))

리뷰 총 개수 : 60


In [22]:
selector = '.eCPGL > .YeINN'
elements = soup.select(selector)
elements

[<li class="YeINN"><div class="Lia3P"><a class="DqSJm" href="https://m.place.naver.com/my/5bf973370c2fcde29f53d3e5/review?v=2" role="button" target="_blank"><div class="place_thumb"><img alt="프로필" class="sKXBJ" height="38" src="https://m-place.pstatic.net/assets/shared/images/icon_default_profile.png" width="38"/></div></a><a class="Hazns" href="https://m.place.naver.com/my/5bf973370c2fcde29f53d3e5/review?v=2" role="button" target="_blank"><div class="sBWyy">네루3682</div><div class="Qde7Q"><span class="P1zUJ">리뷰 <!-- -->1382</span><span class="P1zUJ">사진 <!-- -->246</span></div></a><a class="wFqB7" href="#" role="button">팔로우</a></div><div class="Mmpzx jQTnr"><div class="ngGKH"><div class="flicking-viewport" style="user-select: none; -webkit-user-drag: none; touch-action: pan-y;"><div class="flicking-camera" style="transform: translate(0px);"><div class="ODalI"><a class="v3j4Z" href="#" role="button"><div class="K0PDV _img fKa0W" style='width:100%;height:100%;background-image:url("https:/

In [10]:
try: 
    one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
    review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
    print('리뷰 총 개수 : '+str(review_num)) 
    
    # 모든 리뷰에 대해서 정보 수집
    for i in range(len(one_review)): 
        
        # user url
        user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
        print('user_url = '+user_url) 
        
        # user url로부터 user code 뽑아내기
        user_code = re.findall(r"my/(\w+)", user_url)[0]
        print('user_code = '+user_code) 
        
        # review 1개에 대한 id 만들기 
        res_code = re.findall(r"restaurant/(\d+)", thisurl)[0] 
        review_id = str(res_code)+"_"+user_code
        print('review_id = '+review_id) 
        
        # rating, 별점 
        rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
        print('rating = '+rating) 
        
        ## 주의
        ## 사진 리뷰 유무에 따라 날짜 파싱코드가 다르기 때문에
        ## case 별로 처리해줘야 함
        ## ('span', attrs = {'class':'_3WqoL'}) 
        ## 사진 없는 경우 : 총 6개 중 4번째 
        ## 사진 있는 경우 : 총 5개 중 3번째 
            
        # date 
        # 사진 리뷰 없음 
        if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5: 
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text 

        elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6: 
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text

        else: 
            date = ""
        
        print('date = '+date) 

        
        # review 내용
        try: 
            review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
        except: # 리뷰가 없다면
            review_content = "" 
        print('리뷰 내용 : '+review_content) 
        

        # image 내용
        sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'}) 
        
        if (sliced_soup != None): 
            sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'}) 
            
            try: 
                img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpeg)', str(sliced_soup))[0] 
            
            except : 
                if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0): 
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0): 
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=l&size=800x800&src='+re.findall(r'src=(.*png)', str(sliced_soup))[0]
                else : 
                    img_url = ""
        
        else: 
            img_url = "" 
        
        
        print('이미지 url : '+img_url) 
        print('----------------------------------------------') 
        print('\n') 
        
        
        # 리뷰정보 
        # user_review_id
        user_review_id[restaurant_name][user_code] = review_id 
        
        # review_json 
        review_json[restaurant_name][review_id] = review_content 
        
        # image_json 
        image_json[restaurant_name][review_id] = img_url 
        
        # rating_df_list 
        naver_review = user_code, restaurant_name, rating, date
        rating_list.append(naver_review) 

# 리뷰가 없는 경우        
except NoSuchElementException: 
    none_review = "네이버 리뷰 없음" 
    print(none_review)
    review_num = 0 
    
    # 리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review 
    
    # rating_df_list
    naver_review = user_code, restaurant_name, none_review, none_reivew
    rating_list.append(naver_review)

리뷰 총 개수 : 0
